In [25]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline

# parse the ResultOrders into a table
with open('resultorders.json','r') as ordersjson:
    orders = json.load(ordersjson)
column_names = []
first_item = list(orders.keys())[0]
for v in orders[first_item]:
    if not v in 'fills':
        column_names.append(v)
column_names.append('cash')

table = []
for k, v in orders.items():
    selected_rows = []
    for column in v:
        if column == 'fills':
            total_cash = 0
            for fill in v['fills']:
                total_cash += fill['qty'] * fill['price'] * .01
        else:
            selected_rows.append(v[column])
    selected_rows.append(total_cash)
    table.append(selected_rows)

df = pd.DataFrame(table, columns=column_names)
df['ts'] = pd.to_datetime(df['ts'])

# Load the quotes dataframe
quotes = pd.read_json('ResultQuoteSocket.json')
quotes['quoteTime'] = pd.to_datetime(quotes['quoteTime'])
quotes['lastTrade'] = pd.to_datetime(quotes['lastTrade'])
last = quotes["last"].iloc[-1]
if last is None:
    last = 0

In [26]:
def update_open_orders(orders):
    """Loop through my orders and return positionSoFar, cash, and expectedPosition."""
    position_so_far = 0
    cash = 0
    expected_positions = 0
    for y in orders:
        total_filled = 0
        total_cost = 0
        x = orders[y]            
        # print "\n%r" % (x)
        for fill in x['fills']:
            total_filled += fill['qty']
            total_cost += fill['qty'] * fill["price"]
        qty = x["qty"] + total_filled  # x['qty'] becomes 0 if the order is cancelled. so we just add that to how much is filled already
        direction = x["direction"]
        if direction == "sell":
            total_filled = total_filled * -1
            total_cost = total_cost * -1
            qty = qty * -1
        position_so_far += total_filled
        expected_positions += qty            
        cash += total_cost * (-.01)  # -.01 because we are getting the correct unit
    return position_so_far, cash, expected_positions



In [27]:
"""
for account in df['account'].unique():
    acc_trades = df[df['account'] == account]
    if (len(acc_trades) > 5):
        #player_orders = {k: v for k, v in orders.items() if v['account'] == account}
        #positionSoFar, cash, expectedPos = update_open_orders(player_orders)
        #nav = cash + positionSoFar * last * (.01)
        # bought_positions = acc_trades[acc_trades['direction'] == 'buy']['totalFilled']
        # sold_positions = acc_trades[acc_trades['direction'] == 'sell']['totalFilled']        
        # positionSoFar = bought_positions - sold_positions        
        # print "\nAccount: %r\nPosition: %r\nCash: %r\nNAV: %r\nNumber of orders: %r" % (account, positionSoFar, 
        #                                                                                 cash, nav, len(player_orders))
        pivoted = pd.pivot_table(acc_trades[['ts','direction','price']], 
                                     values='price', 
                                     columns='direction', index='ts')
        pivoted.plot(figsize= (15,5))
        plt.title('Trade Price over Time for %s' %(account))
        plt.show()
"""

'\nfor account in df[\'account\'].unique():\n    acc_trades = df[df[\'account\'] == account]\n    if (len(acc_trades) > 5):\n        #player_orders = {k: v for k, v in orders.items() if v[\'account\'] == account}\n        #positionSoFar, cash, expectedPos = update_open_orders(player_orders)\n        #nav = cash + positionSoFar * last * (.01)\n        # bought_positions = acc_trades[acc_trades[\'direction\'] == \'buy\'][\'totalFilled\']\n        # sold_positions = acc_trades[acc_trades[\'direction\'] == \'sell\'][\'totalFilled\']        \n        # positionSoFar = bought_positions - sold_positions        \n        # print "\nAccount: %r\nPosition: %r\nCash: %r\nNAV: %r\nNumber of orders: %r" % (account, positionSoFar, \n        #                                                                                 cash, nav, len(player_orders))\n        pivoted = pd.pivot_table(acc_trades[[\'ts\',\'direction\',\'price\']], \n                                     values=\'price\', \n           

In [28]:
# df.loc[(df['account'] == 'ISS83432725') & (df['direction'] == 'buy')]

In [49]:
"""
acc_trades = df[df['account'] == 'ISS83432725']
acc_trades.loc[acc_trades.direction == 'buy', 'cash'] *= -1
acc_trades.loc[acc_trades.direction == 'sell', 'totalFilled'] *= -1
acc_trades[['totalFilled','cash','direction']].groupby('direction').head()
"""
acc_trades = df
acc_trades.loc[acc_trades.direction == 'buy', 'cash'] *= -1
acc_trades.loc[acc_trades.direction == 'sell', 'totalFilled'] *= -1
acc_group = acc_trades[['totalFilled','cash','direction','account']].groupby('account').sum()
acc_group['NAV'] = acc_group['totalFilled'] * (last * .01) + acc_group['cash']
acc_group.sort_values(by = 'NAV',ascending = False).head(5)

,totalFilled,cash,NAV
account,,,
MPB97230146,-7243,449184.47,5188.57
CES45554999,38,-2615.71,-286.31


In [30]:
# combined = pd.merge(df, quotes, how = 'outer', left_on='ts', right_on = 'lastTrade')
# combined.dtypes

In [31]:
#combined.to_excel("combined.xlsx")